In [7]:
!pip install pyspark
!pip install -U -q PyDrive
!pip install pyngrok
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u412-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [8]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [9]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [10]:
# create the session
conf = SparkConf()
conf.set("spark.ui.port", "4050")
conf.setAppName("DIS-lab-2")
conf.setMaster("local[*]")
conf.set("spark.driver.memory", "2G")
conf.set("spark.driver.maxResultSize", "2g")
conf.set("spark.executor.memory", "1G")
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [11]:
import json

data_list = []
with open('data_processes.json') as json_file:
   data_list = json.load(json_file)
#spark_data_list = spark.read.json("data_processes.json")


In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

rdd = sc.parallelize(data_list) # data will be operated in parallel


In [13]:
def f(x): print(x)
rdd.foreach(f)

In [14]:
a = rdd.count()
print(a, " ",len(data_list))

8048   8048


In [42]:
filtered_rdd = rdd#.filter(lambda x: x['action'] == "Request")
def f(x):
  list_aux = []
  depth = 0
  for e in x:
    i = 0
    #list_aux.append(e)

    for j in e:
      if i == 2:
        if j == "Request":
          depth += 1
        elif j == "Response":
          depth -= 1
      elif i == 0:
          list_aux.append((j, depth))
      i+=1
  return list_aux

mapped_rdd = filtered_rdd.map(lambda x: (x['process_id'], [x['state_from'], x['state_to'], x['action']])).groupByKey().mapValues(f)

result = mapped_rdd.collect()

for process_id, transitions in result:
    print(f"Process ID: {process_id}")
    for transition in transitions:
        print(f"  {transition}")




Se han truncado las últimas 5000 líneas del flujo de salida.
  ('S3', 2)
  ('S1', 1)
Process ID: process859
  ('user', 0)
  ('S1', 1)
  ('S2', 2)
  ('S7', 3)
  ('S2', 2)
  ('S1', 1)
Process ID: process860
  ('user', 0)
  ('S1', 1)
  ('S2', 2)
  ('S7', 3)
  ('S2', 2)
  ('S1', 1)
Process ID: process863
  ('user', 0)
  ('S1', 1)
  ('S3', 2)
  ('S5', 3)
  ('S3', 2)
  ('S1', 1)
  ('S2', 2)
  ('S7', 3)
  ('S2', 2)
  ('S1', 1)
Process ID: process864
  ('user', 0)
  ('S1', 1)
  ('S2', 2)
  ('S7', 3)
  ('S2', 2)
  ('S1', 1)
Process ID: process865
  ('user', 0)
  ('S1', 1)
  ('S2', 2)
  ('S7', 3)
  ('S2', 2)
  ('S1', 1)
  ('S3', 2)
  ('S5', 3)
  ('S3', 2)
  ('S1', 1)
Process ID: process866
  ('user', 0)
  ('S1', 1)
  ('S2', 2)
  ('S7', 3)
  ('S2', 2)
  ('S1', 1)
Process ID: process871
  ('user', 0)
  ('S1', 1)
  ('S3', 2)
  ('S5', 3)
  ('S3', 2)
  ('S1', 1)
  ('S2', 2)
  ('S7', 3)
  ('S2', 2)
  ('S1', 1)
Process ID: process872
  ('user', 0)
  ('S1', 1)
  ('S3', 2)
  ('S5', 3)
  ('S3', 2)
  ('S1'

In [ ]:
# Filtrar los datos para quedarnos solo con las acciones "Request"
filtered_rdd = rdd.filter(lambda x: x['action'] == "Request")

# Transformar los datos en la estructura deseada (process_id, [state_from, state_to])
mapped_rdd = filtered_rdd.map(lambda x: (x['process_id'], [x['state_from'], x['state_to']]))

# Agrupar por process_id y combinar las listas de transiciones
grouped_rdd = mapped_rdd.groupByKey().mapValues(lambda x: [item for sublist in x for item in sublist])

# Función para convertir lista de transiciones a un contador de transiciones
def transitions_counter(transitions):
    return Counter(transitions)

# Contar las transiciones para cada proceso
transition_counts_rdd = grouped_rdd.mapValues(transitions_counter)

# Definir una función de reducción para agrupar procesos similares si tienen al menos tres transiciones iguales
def reduce_similar_processes(a, b):
    # Comparar los dos contadores y verificar si tienen al menos tres transiciones iguales
    common_transitions = a & b  # Intersección de los contadores
    if sum(common_transitions.values()) >= 3:
        return a + b  # Unir los contadores si tienen al menos tres transiciones iguales
    return a

# Aplicar reduceByKey para agrupar procesos con transiciones similares
similar_processes_rdd = transition_counts_rdd.map(lambda x: (frozenset(x[1].items()), [x[0]])) \
                                             .reduceByKey(lambda a, b: a + b) \
                                             .filter(lambda x: len(x[1]) > 1)  # Filtrar para obtener solo aquellos con al menos dos procesos

# Recoger el resultado
result = similar_processes_rdd.collect()

# Imprimir los procesos similares
for common_transitions, process_ids in result:
    print("Common transitions:")
    for transition, count in common_transitions:
        print(f"  {transition}: {count}")
    print("Process IDs:")
    for process_id in process_ids:
        print(f"  {process_id}")


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 90.0 failed 1 times, most recent failure: Lost task 1.0 in stage 90.0 (TID 173) (b2baacc3a8a9 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python3.10/d

In [ ]:
filtered_rdd = rdd#.filter(lambda x: x['action'] == "Request")
def f(x):
  list_aux = []
  depth = 0
  for e in x:
    i = 0
    for j in e:
      if i == 2:
        if j == "Request":
          depth += 1
        elif j == "Response":
          depth -= 1
      else:
          list_aux.append((j, depth))
      i+=1
  return list_aux

mapped_rdd = filtered_rdd.map(lambda x: (x['process_id'], [x['state_from'], x['state_to'], x['action']])).groupByKey().mapValues(f)

result = mapped_rdd.collect()

for process_id, transitions in result:
    print(f"Process ID: {process_id}")
    for transition in transitions:
        print(f"  {transition}")


